In [1]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=d5fdfeb4675c3c51556aacbfe862e6859b4cf5a4b1519041409ead916d4cc582
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
import warnings
warnings.simplefilter(action='ignore')

import os
import requests
import joblib

import numpy
import pandas

import matplotlib.pyplot as plt
import scipy.stats as stats
import tensorflow_decision_forests as tfdf

In [3]:
from catboost import CatBoostRegressor ,Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.svm import SVR

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectFromModel

from sklearn.impute import SimpleImputer
import sklearn.linear_model as linear_model
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [4]:
train = pandas.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv').drop(columns=['ID'])
test = pandas.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv').drop(columns=['ID'])

In [5]:
from lifelines import KaplanMeierFitter

from lifelines.utils import concordance_index as ci_lifelines

def calculate_survival_probabilities(df, time_col, event_col):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    return kmf.survival_function_at_times(df[time_col]).values

def preprocess_survival_data(df, time_col='efs_time', event_col='efs'):
    df['target'] = calculate_survival_probabilities(df, time_col, event_col)
    df.loc[df[event_col] == 0, 'target'] -= 0.2  # Adjust for censored data
    return df

train = preprocess_survival_data(train)

In [6]:
encoder = LabelEncoder()
normalize = SimpleImputer(strategy='mean')

In [7]:
for i in zip(train.columns,train.dtypes):
    if i[1]=='O':
        train[i[0]] = train[i[0]].fillna('Unknown')
        train[i[0]]=encoder.fit_transform(train[i[0]].to_numpy().reshape(-1,1))
    else:
        train[i[0]] = train[i[0]].fillna(0)
        #train[i[0]]=normalize.fit_transform(numpy.array(train[i[0]]).reshape(-1,1))
for i in zip(test.columns,test.dtypes):
    if i[1]=='O':
        test[i[0]] = test[i[0]].fillna('Unknown')
        test[i[0]]=encoder.fit_transform(test[i[0]].to_numpy().reshape(-1,1))
    else:
        test[i[0]] = test[i[0]].fillna(0)
        #test[i[0]]=normalize.fit_transform(numpy.array(test[i[0]]).reshape(-1,1))

In [8]:
train_x = train.drop(columns=['efs', 'efs_time', 'target'])

train_y = train['target']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.1, random_state=100)

In [10]:
catboost = CatBoostRegressor(grow_policy='Depthwise',
                             iterations=1000,
                             eval_metric='RMSE',
                             random_state=0,
                             verbose=100)



cat_features = list(train_x.select_dtypes(include=['object', 'category']).columns)
    
train_pool = Pool(X_train, y_train, cat_features=cat_features)
val_pool = Pool(X_test, y_test, cat_features=cat_features)
        
catboost.fit(train_pool, eval_set=(val_pool), verbose=50, early_stopping_rounds=100)

Learning rate set to 0.084886
0:	learn: 0.2582909	test: 0.2559741	best: 0.2559741 (0)	total: 76ms	remaining: 1m 15s
50:	learn: 0.2314966	test: 0.2365273	best: 0.2365273 (50)	total: 897ms	remaining: 16.7s
100:	learn: 0.2233461	test: 0.2337668	best: 0.2337668 (100)	total: 1.65s	remaining: 14.7s
150:	learn: 0.2144036	test: 0.2325576	best: 0.2325576 (150)	total: 2.37s	remaining: 13.3s
200:	learn: 0.2057776	test: 0.2321114	best: 0.2320361 (189)	total: 3.06s	remaining: 12.2s
250:	learn: 0.1987167	test: 0.2320816	best: 0.2319514 (213)	total: 3.75s	remaining: 11.2s
300:	learn: 0.1923142	test: 0.2316385	best: 0.2316327 (288)	total: 4.42s	remaining: 10.3s
350:	learn: 0.1865115	test: 0.2317754	best: 0.2315994 (322)	total: 5.09s	remaining: 9.42s
400:	learn: 0.1816362	test: 0.2322677	best: 0.2315994 (322)	total: 5.76s	remaining: 8.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2315993575
bestIteration = 322

Shrink model to first 323 iterations.


In [11]:
cat_pre_test=catboost.predict(test)
print(f'Concordance index (lifelines): {ci_lifelines(y_test, catboost.predict(X_test))}')

Concordance index (lifelines): 0.6434623996950867


In [12]:
id = pandas.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')['ID']
test_predictions = (cat_pre_test)
#((RandomF_pre_test/Fold)+(LGBM_pre_test/Fold)+(xgb_pre_test/Fold)+(cat_pre_test/Fold))/4
#model.predict(new_test_data)
#((RandomF_pre_test/10)+(LGBM_pre_test/5)+(xgb_pre_test/5)+(cat_pre_test/5))/4
test_predictions

array([0.34199803, 0.87639948, 0.34063601])

In [13]:
submission = pandas.DataFrame({
    'ID': id.values,
    'prediction': test_predictions,
})
submission

,ID,prediction
0,28800,0.341998
1,28801,0.876399
2,28802,0.340636


In [14]:
submission.to_csv('submission.csv', index = False)